<a href="https://colab.research.google.com/github/jardel-alves0314/analise-de-dados-puc-2023/blob/main/mvp_sprint_1/01_notebook_auxiliar_classe_extrator_jardel_dos_anjos_alves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***CLASSE EXTRATOR***

## Notebook responsável pela extração de dados das ações negociadas no Brasil pela IBOVESPA

# **Importação das Bibliotecas**

In [ ]:
import pandas  as pd
import numpy   as np
import seaborn as sea
import scipy   
import requests
from datetime import datetime

from bs4       import BeautifulSoup
import yfinance as yf

from google.colab import drive

from abc import ABC, abstractmethod

--------------------------------------------------------------------------------

##**Classe BuscaAcao**

A classe BuscaAcao encapsula as funcionalidades básicas da biblioteca Yahoo Fincnace.

In [ ]:
class BuscaAcao:
  def __buscar_acao_yahoo_finance(self,acao,inicio,fim):
    try:
      #recebe as cotações da ação (ticker)
      df = yf.download(acao,start=inicio,end=fim)

      #cria a coluna Ticker no dataframe e atribui o nome da ação, a biblioteca não traz o nome da ação no dataframe
      df["ticker"] = acao

      #cria a coluna Tading Date, e atribui a ela o valor do índice usado no dataframe, nesse caso é a data de fechamento da ação
      df["trading_date"] = df.index

      df = df.reindex(columns = ["trading_date", "ticker", "Open", 	"High", 	"Low", 	"Close", 	"Adj Close", 	"Volume"])
      df = df.rename(columns={"Open":"open", 	"High":"high", 	"Low":"low", 	"Close":"close", 	"Adj Close":"adj_close", 	"Volume":"volume"})
      # df.set_index(["trading_date","ticker"],inplace = True)
      return df
    except:
      print(f"não foi possível reuperar a cotação da ação: {acao} no período de {inicio} até {fim}")
    
    return None
    
  def buscar_acao(self,acao,inicio,fim):
    resultado = self.__buscar_acao_yahoo_finance(acao,inicio,fim)
    return resultado
                                                  

--------------------------------------------------------------------------------

##**CLASSE WebScraper**

A classe WebScraper recebe uma url e faz o "parse" da página html extraindo os dados da pesquisa.

A saída é uma lista com o resultado da pesquisa.

In [ ]:
class WebScraper:
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
  def htmlParcer(self, url):
    response = requests.get(url,self.headers).content
    soup  = BeautifulSoup(response,  'html.parser')
    tr = soup.find('table').findAll("tr")[1:]
    resultado = []
    for it in tr:
      td = it.findAll("td")
      resultado.append(td[0].text)

    resultado.sort()
    
    return resultado

--------------------------------------------------------------------------------

##**Classe Abstrata Extrator**

In [ ]:
class Extrator(ABC):
  def __salvar_no_google_colab(self, df, nome, diretorio, modo):
     if df is not None:
       if not df.empty:
         df.to_csv(f"/content/drive/My Drive/{diretorio}/{nome}", mode = modo, index = False)
         print("arquivo salvo com sucesso!!!")
       else:
         print("nao foi possivel salvar - dataframe está vazio!")
    

  def salvar(self, repositorio, df, nome, diretorio, modo = "w"):
    if repositorio == "colab":
      self.__salvar_no_google_colab(df, nome, diretorio, modo)

--------------------------------------------------------------------------------

##**Classe ExtratorDeAcoes**

A classe ExtratorDeCotacoes recebe uma url que contém a lista das ações e gera um arquivo por período para ser usado depois com as cotações.

In [ ]:
class ExtratorDeAcoes(Extrator):
  def __init__(self):    
    self.scraper  = WebScraper()    
    drive.mount("/content/drive",force_remount=True)
    

  def extrair_salvar_csv(self, url, repositorio, nome_arquivo, diretorio):
    lista_de_nome_das_acoes = self.scraper.htmlParcer(url)

    df_acoes   = pd.DataFrame(lista_de_nome_das_acoes,columns = ["ticker"])

    df_acoes["ticker"] = df_acoes["ticker"].apply(lambda x:x+".SA")

    df_acoes = df_acoes.reset_index(drop=True)    

    df_acoes = df_acoes.reindex(columns = ["ticker"])
    
   

    if df_acoes is not None:
      if not df_acoes.empty:
        super().salvar(repositorio, df_acoes, nome_arquivo, diretorio, modo = "w")

  

--------------------------------------------------------------------------------

## **Classe ExtratorDeCotacoes**

A classe ExtratorDeCotacoes recebe uma url que contém a lista das ações e através dessa lista busca a cotação delas num determinado período de tempo.


In [ ]:
class ExtratorDeCotacoes(Extrator):
  def __init__(self):
    self.buscador = BuscaAcao()
    self.scraper  = WebScraper()
    
    drive.mount("/content/drive",force_remount=True)
    

  def extrair_salvar_csv(self, url, data_inicio, data_fim, repositorio, nome_arquivo, diretorio, tamanho = -1):
    df_acoes   = None
    df         = None
    quantidade = 0

    lista_de_nome_das_acoes = self.scraper.htmlParcer(url)
    for acao in lista_de_nome_das_acoes:
      print(acao)
      quantidade += 1
      if quantidade > tamanho and tamanho > 0:
        break

      df = self.buscador.buscar_acao(f"{acao}.SA", data_inicio, data_fim)

      if df is not None:
        if not df.empty:
          if df_acoes is not None:
            df_acoes = pd.concat([df_acoes,df])
          else:
            df_acoes = df
            
      super().salvar(repositorio, df_acoes, nome_arquivo, diretorio, modo = "w")  


## **Classe ExtratorOutrosIndices**

A classe ExtratorOutrosIndices recebe o código de um índice econômico e busca a cotação dele num determinado período de tempo.


In [ ]:
class ExtratorOutrosIndices(Extrator):
  def __init__(self):
    self.buscador = BuscaAcao()
    self.scraper  = WebScraper()
    
    drive.mount("/content/drive",force_remount=True)
    

  def extrair_salvar_csv(self, indice_economico, data_inicio, data_fim, repositorio, nome_arquivo, diretorio):
    df = self.buscador.buscar_acao(indice_economico, data_inicio, data_fim)
    
    if df is not None:
        if not df.empty:          
          super().salvar(repositorio, df, nome_arquivo, diretorio, modo = "w")     


#**Execução da extração das ações e suas cotações**

Fonte: https://www.dadosdemercado.com.br/bolsa/acoes

Período: 2018-01-01 a 2022-12-31 


In [ ]:

url = "https://www.dadosdemercado.com.br/bolsa/acoes"

repositorio = "colab"

nome_arquivo_acoes = "lista_de_acoes_2023_04_11.csv"

diretorio = "Colab Notebooks/PUC/analise-de-dados-puc-2023"

extrator_acoes = ExtratorDeAcoes()

extrator_acoes.extrair_salvar_csv(url, repositorio, nome_arquivo_acoes, diretorio)


Mounted at /content/drive
arquivo salvo com sucesso!!!


In [ ]:

url = "https://www.dadosdemercado.com.br/bolsa/acoes"

data_inicio = "2018-01-01"

data_fim = "2022-12-31"

repositorio = "colab"

nome_arquivo_cotacoes = "lista_de_cotacoes_2023_04_11.csv"

diretorio = "Colab Notebooks/PUC/analise-de-dados-puc-2023"

extrator_cotacoes = ExtratorDeCotacoes()

extrator_cotacoes.extrair_salvar_csv(url, data_inicio, data_fim, repositorio, nome_arquivo_cotacoes, diretorio)


Mounted at /content/drive
AALR3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
ABCB4
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
ABEV3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
AERI3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
AESB3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
AFLT3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
AGRO3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
AGXY3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
AHEB3
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
AHEB5
[*********************100%******

In [ ]:
# drive.mount("/content/drive",force_remount=True)

# nome_arquivo_final = "lista_cotacoes_acoes_2023_04_07.csv"

# df_acoes =  pd.read_csv(f"/content/drive/My Drive/{diretorio}/{nome_arquivo_acoes}")

# df_cotacoes =  pd.read_csv(f"/content/drive/My Drive/{diretorio}/{nome_arquivo_cotacoes}")

# df = pd.merge(df_acoes, df_cotacoes, on=["trading_date", "ticker"], how='left')

# df

##teste ibovesp

In [ ]:

url = "https://www.dadosdemercado.com.br/bolsa/acoes"

data_inicio = "2018-01-01"

data_fim = "2022-12-31"

repositorio = "colab"

nome_arquivo_cotacoes = "lista_de_indices_bovespa_2023_04_11.csv"

diretorio = "Colab Notebooks/PUC/analise-de-dados-puc-2023"

extrator_ibovespa = ExtratorOutrosIndices()

extrator_ibovespa.extrair_salvar_csv("^BVSP",data_inicio, data_fim, repositorio, nome_arquivo_cotacoes, diretorio)


Mounted at /content/drive
[*********************100%***********************]  1 of 1 completed
arquivo salvo com sucesso!!!
